* 팀명 : Data Freaks
* 팀원 : 강찬, 손현우, 천용재, 이대근
* 내용 : 안양시 내 80개의 IOT센서 위치 분석과 40개의 미세먼지 센서 위치 분석

## 도시데이터 수집을 위한 IOT센서 설치 위치 분석
### 분석순서
* 1. 안양시의 최남단 서쪽에서부터 최북단 동쪽까지, 위경도를 일정량 증가시켜가며 예비측정소가 될 점을 찍는다.(이하 '점'이라 칭함)
* 2. 점마다 주변에 있는 요소에 따라 점수를 매긴다. 이 단계가 끝나면 바둑판처럼 점이 찍히고 각 점에 점수가 부여된다.

> 이 때 안양시의 외부에도 점을 찍게되는데 데이터파일은 안양시에만 관련된 것이므로 자연스럽게 소거된다.
> 만약을 위해 점의 개수를 필요한 것 보다 많이 추출하고 점수가 낮은 하위점들을 덜어낸다

> 점의 간격과 점에 포함될 요소의 범위를 필요시 조절하여야 한다.

> 점수는 '거리'와, 요소의 크기가 있다면(ex)학교의 인원) 그 크기에 따라 0부터 1까지 부여해서 곱하여 부여하고, 요소가 없다면 거리에 따라서만 부여한다.

* 3. 필요한 점들만 선별한다. 점을 찍을때와 같은 방법으로 일정 간격마다 원을 그리고, 이 원에서 점수가 제일 높은 점 하나만을 남긴다, 이 점이 최종적인 측정소의 위치가 된다.

> 원의 크기를 조절하여 인접한 원끼리 겹치는 정도를 조절할 수 있고, 그에 따라 선별된 점이 중복되는 정도를 줄일 수 있다.

> 원의 간격을 조절하여 선별되는 점의 개수를 조절할 수 있다.

* 4. 미세먼지 데이터에 대해서도 시행

* 5. 데이터 시각화

### ##는 큰 관점에서의 역할, 기능을 상세하는 주석
### #는 기술적인 의미를 자세히 상세하는 주석

In [42]:
##함수 및 라이브러리 모음

import pandas as pd ## 데이터파일을 다루기 위해 필요한 라이브러리
import math ## 위경도를 거리로 계산하기 위해 필요한 라이브러리

PLimitDistance = 500 #점(예비 측정소 설치지점)에서 요소를 포함할 범위 (미터) # 필요시 수정하여야 한다
PDis = 0.002 #점간의 간격 0.0025 > 약 225 ~ 300m # 필요시 수정

def DisScaler(x): ##점과 요소의 거리에 따라 스케일링. 범위의 끝에 있을 시 0점
    for i in range(0,len(x)):
        x[i]=((PLimitDistance-x[i])/PLimitDistance)
    return

def MMScaler(x): ##요소의 크기에 따라 0~1로 스케일링하여 돌려주는 함수 (ex 인구 2, 4, 6 >0, 0.5, 1)
    #x는 요소에 대한 list
    mx = max(x)
    mn = min(x)  
        
    if mx == 0:
        return
    if mx - mn == 0: #우연히 데이터가 같은 것 끼리만 모였을 때는 요소를 전부 1로 주어 거리만 점수에 포함되게 한다
        for i in range(0,len(x)):
            x[i] = 1 
        return

    for i in range(0,len(x)): #min max scaling
        x[i] = (x[i]-mn)/(mx-mn)
    return
    
def rad(x): ##GetDistance1 필요 함수
    return (x*3.14159/180.0)

def GetDistance1(p1x, p1y,p2x,p2y):  ##경위도 간 거리 구하는 함수, x=Lon=경도 , y=Lat=위도
    dLat = rad((p2y-p1y));
    dLon = rad((p2x-p1x));
    
    p1y = rad(p1y);
    p2y = rad(p2y);

    a = math.sin(dLat/2) * math.sin(dLat/2) + math.sin(dLon/2) * math.sin(dLon/2) * math.cos(p1y) * math.cos(p2y);
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a));
    radius = 6371;
    dDistance = radius * c;
    dDistance*=1000;

    return dDistance;


def PandS(DF,Col=3): #Point and Score
    ##안양시 내에서 점(예비 설치소)을 일정간격으로 찍고, 점마다 데이터 파일을 기반으로 점수를 매기는 함수

    #요소가 데이터의 '존재'만 다룰 시 (=val열이 없거나 1로만 존재할 시) Col=0으로 둔다. 
    #val이 1만 있을 시 MMScaler를 쓸 수 없기 때문. val이 0과 1로만 상존하는 경우는 관계 없음
    
    result = pd.DataFrame(columns=["lat", "lon","score"]) #결과를 저장해서 return할 dataframe
    Blat = 37.366125 #안양시 시작지점 위경도
    Blon = 126.871798 
    list1 = [] # lat
    list2 = [] # lon
    list3 = [] # score
        
    while(True):
        scr = 0 #점수로 쓸 변수 선언
        if Blat >= 37.448805: #안양시 내에서 위도(가로) 증가, 범위 끝 도달 시 경도 증가 후 위도 초기화
            Blat = 37.366125
            Blon += PDis #경도를 간격만큼 벌림
            
        L4S = [] # 요소의 크기(인구밀도 등)에 따른 스케일링을 위한 리스트 List for Scale
        L4D = [] # 거리 스케일링을 위한 리스트 List for Distance
                
        ##점의 범위안에 있는 요소를 저장하는 반복문
        for i in range(0, len(DF)): 
            dis = GetDistance1(Blon,Blat,DF["lon"][i], DF["lat"][i]) #점과 요소의 거리 계산후 저장
            if dis <= PLimitDistance: #원 안에 포함되는 요소와 거리 모음
                L4D.append(dis)
                if Col:
                    L4S.append(DF.iloc[i,Col]) #Col==0일 시 L4S 필요 없음 (거리만 점수로 계산)
                
        ##요소에 따라 거리로만 점수부여, 혹은 (거리*요소의 크기)로 점수 부여        
        if len(L4D): #원에 포함된 요소가 없으면 점수를 계산하지 않음
            if len(L4D)==1 or not Col: #원에 한개의 요소만 포함됐을 시 or Col==0일 시에는 거리만 계산(스케일링 불가)
                DisScaler(L4D)
                for i in range(0, len(L4D)):  #스케일하여 점수 합산
                    scr += L4D[i]
                
            else:#한 개 이상 포함됐을 시 거리와 요소 각각 스케일링
                MMScaler(L4S)
                DisScaler(L4D)
                for i in range(0, len(L4D)): # 스케일 후 점수계산 ('요소*거리' > 요소 수만큼 반복)
                    scr +=L4S[i]*L4D[i]
        
        list1.append(Blat) ##찍은 점과 부여한 점수 기록
        list2.append(Blon)
        list3.append(scr)
        
        if Blon >= 126.980480: ##점을 바꿔가며 반복
            break #안양시 끝까지 도달했을 시 중단
        Blat = Blat + PDis
    
    result["lat"] = list1 #데이터프레임으로 옮겨담기
    result["lon"] = list2
    result["score"] = list3
    return result 

def addScore(df1, df2): ##데이터별로 계산된 점수를 합산하는 함수
    dfCopy = df1.copy() #합산할 결과를 저장하기 위해 데이터프레임 카피
    dfCopy.iloc[:,2]=df1.iloc[:,2]+df2.iloc[:,2] #score 합산 후 리턴
    return dfCopy

def decideLoc(df, CDis=0.008): ##점마다 매긴 점수에 기반하여 최종 점(측정소)를 선별하는 함수
    ## 분석순서의 3번에 해당하는 단계
    #CDis # 원 중심간의 간격 (0.006 = 530~600m, 0.01 = 880~950) # 필요시 수정
    
    Blat = 37.366125 #안양시 시작지점 위경도
    Blon = 126.871798   
    
    Rad = (GetDistance1(Blon, Blat, Blon+CDis,Blat))*4/5 # 원의 반지름 = 중심간격의 80퍼센트 # 필요시 수정
    
    finalDF = pd.DataFrame(columns=["lat", "lon","score"]) #최종적으로 리턴할 위치를 위한 데이터프레임
    
    count = 0 #최종목록(finalDF)의 개수
    print("원의 개수(중복포함)") #진행상황 파악을 위한 print
    
    while(True): ##PandS와 같은 방식의 진행방법으로, 선별한 점을 dataframe에 계속 담는다
        if Blat >= 37.448805: #안양시 내에서 위도(가로) 증가, 범위 끝 도달 시 경도 증가 후 위도 초기화
            Blat = 37.366125
            Blon += CDis
            
        tempDF =  pd.DataFrame(columns=["lat", "lon","score"]) #원안에 포함된 점을 담아두는 임시 DF
        a = 0 #임시변수, 원 안에 포함된 점들의 개수
        
        ## 원 안에 포함되는 점 들을 tempDF에 담는 반복문
        for i in range(0, len(df)): 
            dis = GetDistance1(Blon,Blat,df["lon"][i], df["lat"][i]) #원 중심과 점의 거리 계산
            if dis <= Rad and df.iloc[i,2] != 0: #원 안에 포함되는 점이 0점이 아닐 시 모음
                tempDF.loc[a] = df.loc[i]
                a += 1
         
        ## 원 안에 포함된 점 중 점수가 가장 높은 점을 리턴할 DF에 저장
        if a: #tempDF가 비었을 시 실행하지 않음
            finalDF.loc[count] = tempDF[tempDF.score==tempDF['score'].max()].iloc[0] #원안의 점 중 점수가 가장 높은 점 최종목록에 추가
            count += 1
            print(count, end='.')
        
        ## 간격만큼 벌려서 원을 다시 그려가며 반복
        if Blon >= 126.980480: #경도 범위 끝에 도달 시 종료, 경도 증가
            break 
        Blat += CDis
        
    return(finalDF.drop_duplicates().sort_values('score',ascending=False))

* 데이터 전처리가 완료된 파일들로 진행한다. 파일명과 경로 유의

In [98]:
busDF = pd.read_csv('13_14_안양시_버스정류장_변환.csv')
kidDF = pd.read_csv('07.안양시_유치원현황_변환.csv')
schoolDF = pd.read_csv('06.안양시_학교현황_변환.csv')
hospitalDF = pd.read_csv('09.안양시_병원정보_변환.csv')
facDF = pd.read_csv('10.안양시_공장등록현황_변환.csv')
facNumDF= pd.read_csv('11.안양시_공장등록현황_격자별_변환.csv')
populDF = pd.read_csv('01.안양시_격자별인구현황(전체)_변환.csv')
subwayDF = pd.read_csv('15.안양시_지하철역_종합정보_변환.csv', encoding='cp949')

fltDF = pd.read_csv('05.안양시_월별_시간대별_유동인구_변환.csv')
fltDF = fltDF[fltDF['value']>600] #데이터가 많아서 실행시간 문제로 데이터 축소, 올바른 결과를 얻으려면 >10~100으로 제한  # >300까지 구동 확인
fltDF.reset_index(drop=True,inplace=True)

* 점수부여단계, 데이터파일별로 PandS 호출
* 시간이 걸리기 때문에 셀을 나눠서 시행

In [6]:
kid = PandS(kidDF,3)
school = PandS(schoolDF,3)

In [7]:
bus = PandS(busDF,6)

In [8]:
subway = PandS(subwayDF,4)

In [9]:
hospital = PandS(hospitalDF,0)

In [101]:
facAir = PandS(facDF,3)

In [102]:
facSound = PandS(facDF,5)

In [14]:
facNum = PandS(facNumDF,3)

In [15]:
popul = PandS(populDF,3)

In [103]:
flt = PandS(fltDF,3)

* 데이터별로 점에 점수를 부여한 것을 합친다.

In [105]:
##1단계 (데이터 + 데이터 = 1차합산데이터)

midScore1 = addScore(kid,school)
midScore2 = addScore(bus, subway)
midScore3 = addScore(hospital,facAir)
midScore4 = addScore(facSound,facNum)
midScore5 = addScore(popul,flt)

##2단계 (1차합산 + 1차합산 = 2차합산)
fScore1 = addScore(midScore1, midScore2)
fScore2_notuse = addScore(midScore3, midScore4) #fScore는 2단계에서 더하고 최종합산에는 쓰지 않는다.
fScore3 = addScore(fScore2_notuse, midScore5)

##최종합산
totalScore = addScore(fScore1, fScore3)

In [106]:
## 점 선별 (분석 순서에서 3번)
finLoc = decideLoc(totalScore)

원의 개수(중복포함)
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.25.26.27.28.29.30.31.32.33.34.35.36.37.38.39.40.41.42.43.44.45.46.47.48.49.50.51.52.53.54.55.56.57.58.59.60.61.62.63.64.65.66.67.68.69.70.71.72.73.74.75.76.77.78.79.80.81.82.83.84.85.86.87.88.89.90.91.92.93.94.95.96.97.98.99.100.101.102.103.104.105.106.107.108.109.110.111.112.113.114.115.

In [107]:
finLoc.reset_index(drop=True,inplace=True) #인덱스 재부여, 인덱스 = 우선순위
finLoc

,lat,lon,score
0,37.390125,126.951798,74.144257
1,37.390125,126.949798,72.934107
2,37.390125,126.953798,52.819490
3,37.434125,126.901798,48.801246
4,37.402125,126.975798,43.584340
...,...,...,...
100,37.418125,126.969798,0.145643
101,37.438125,126.933798,0.132192
102,37.430125,126.893798,0.102254
103,37.366125,126.969798,0.049836


In [108]:
final = finLoc.iloc[:80,:] ##상위 80개 선별
final

,lat,lon,score
0,37.390125,126.951798,74.144257
1,37.390125,126.949798,72.934107
2,37.390125,126.953798,52.819490
3,37.434125,126.901798,48.801246
4,37.402125,126.975798,43.584340
...,...,...,...
75,37.418125,126.933798,2.225627
76,37.438125,126.905798,1.982312
77,37.402125,126.879798,1.496570
78,37.390125,126.901798,1.390052


In [109]:
##csv파일로 저장
final.to_csv('finalSpots.csv') 

* 데이터 시각화

In [110]:
##데이터 시각화를 위한 folium 라이브러리 설치
!pip install folium

In [111]:
##folium을 통한 마커표시
import folium
m = folium.Map(location =  [37.407465 ,126.926139], zoom_start = 15, tiles = 'http://api.vworld.kr/req/wmts/1.0.0/58EEDF9C-7ADA-3459-B779-33088E2A8E34/Satellite/{z}/{y}/{x}.jpeg', attr = '고양시')

folium.TileLayer(tiles = 'http://api.vworld.kr/req/wmts/1.0.0/58EEDF9C-7ADA-3459-B779-33088E2A8E34/Hybrid/{z}/{y}/{x}.png', attr = '고양시', overlay = True).add_to(m)

for index, row in final.iterrows():
    folium.Marker([row['lat'], row['lon']],popup=str(row['score'])).add_to(m) #마커 클릭 시 점수가 표시된다.
    folium.CircleMarker([row['lat'],row['lon']],radius=row['score']*1.5).add_to(m) #점수를 직관적으로 볼 수 있게 원형 마커 추가
m


--------------------


## 미세먼지 센서의 설치 위치 분석

* 여러 데이터를 사용하려 해봤으나, 목적에 제일 부합하는 데이터는 '도로', '교통량', 그리고 이 두가지를 잘 반영한다고 생각되는 '버스정류장' 데이터라 판단

> 링크(도로) 데이터의 경우 처음과 끝에서 교통량은 일정하기 때문에 노드의 처음과 끝 데이터를 사용

* 기타 분석방법은 IOT센서의 설치 위치 분석과 동일

In [112]:
## 사용할 데이터파일 목록
busDF #= pd.read_csv('13_14_안양시_버스정류장_변환.csv')
startDF = pd.read_csv('교통량_끝점.csv')
endDF =pd.read_csv('교통량_시작점.csv')

In [113]:
## 각 데이터파일에 대해 PandS 시행
bus #= PandS(startDF)
start = PandS(startDF)
end = PandS(endDF)

In [114]:
## 점수 합산 후 선별작업 실행
Score1 = addScore(start, end)
totalScore2 = addScore(Score1, bus)

finLoc2 = decideLoc(totalScore2)
finLoc2

원의 개수(중복포함)
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.25.26.27.28.29.30.31.32.33.34.35.36.37.38.39.40.41.42.43.44.45.46.47.48.49.50.51.52.53.54.55.56.57.58.59.60.61.62.63.64.65.66.67.68.69.70.71.72.73.74.75.76.77.78.79.80.81.82.83.84.85.86.87.88.89.90.91.92.93.94.95.96.97.98.99.

,lat,lon,score
27,37.424125,126.905798,30.038994
74,37.402125,126.949798,26.795464
66,37.390125,126.947798,26.523896
80,37.394125,126.959798,26.043922
79,37.382125,126.965798,25.960973
...,...,...,...
83,37.410125,126.961798,0.643597
7,37.386125,126.885798,0.490995
98,37.418125,126.973798,0.422975
47,37.442125,126.917798,0.396779


In [115]:
## 상위 40개 선별, 인덱스 재부여
final2 = finLoc2.iloc[:40,:]
final2.reset_index(drop=True,inplace=True)
final2

,lat,lon,score
0,37.424125,126.905798,30.038994
1,37.402125,126.949798,26.795464
2,37.390125,126.947798,26.523896
3,37.394125,126.959798,26.043922
4,37.382125,126.965798,25.960973
5,37.400125,126.947798,25.841275
6,37.390125,126.931798,24.567264
7,37.398125,126.929798,23.481862
8,37.402125,126.957798,23.464770
9,37.422125,126.905798,23.391089


In [116]:
final2.to_csv("finalSpots2.csv")

In [117]:
m2 = folium.Map(location =  [37.407465 ,126.926139], zoom_start = 15, tiles = 'http://api.vworld.kr/req/wmts/1.0.0/58EEDF9C-7ADA-3459-B779-33088E2A8E34/Satellite/{z}/{y}/{x}.jpeg', attr = '고양시')

folium.TileLayer(tiles = 'http://api.vworld.kr/req/wmts/1.0.0/58EEDF9C-7ADA-3459-B779-33088E2A8E34/Hybrid/{z}/{y}/{x}.png', attr = '고양시', overlay = True).add_to(m)

for index, row in final2.iterrows():
    folium.Marker([row['lat'], row['lon']],icon=folium.Icon(color='green'),popup=str(row['score'])).add_to(m2) #마커 클릭 시 점수가 표시된다.
    folium.CircleMarker([row['lat'],row['lon']],radius=row['score'],color='green').add_to(m2) #점수를 직관적으로 볼 수 있게 원형 마커 추가
m2

* 합쳐서 보여주기

In [118]:
for index, row in final2.iterrows():
    folium.Marker([row['lat'], row['lon']],icon=folium.Icon(color='green'),popup=str(row['score'])).add_to(m) #마커 클릭 시 점수가 표시된다.
    folium.CircleMarker([row['lat'],row['lon']],radius=row['score'],color='green').add_to(m) #점수를 직관적으로 볼 수 있게 원형 마커 추가
m

감사합니다